https://drive.google.com/drive/folders/1Xxk5YOOiUZZ-TCWULDTwUZrhxSGx47QK?usp=drive_link  

above is the dataset path.add it as a shortcut to your drive

Before running this cell, ensure you have installed unidecode with:


In [ ]:
pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 5.1 MB/s eta 0:00:00


function used to display data in tabular form

In [ ]:
import pandas as pd
from IPython.display import display

def display_dataframe(df, num_rows=5):
    """
    Display a DataFrame with enhanced formatting within Jupyter/Colab notebooks.

    Args:
    df (pd.DataFrame): The DataFrame to be displayed.
    num_rows (int): Number of rows to display from the DataFrame.
    """
    # Set display options
    pd.set_option('display.max_columns', None)  # Show all columns
    pd.set_option('display.max_rows', None)  # Show all rows up to the given default
    pd.set_option('display.expand_frame_repr', True)  # Don't squeeze frame horizontally
    pd.set_option('max_colwidth', None)  # Don't truncate column content
    pd.set_option('display.width', 1000)  # Width of the display

    # Display the DataFrame
    print("DataFrame Overview:")
    display(df.head(num_rows))  # Display the first 'num_rows' rows from the DataFrame

importing the necessary packages

In [ ]:
# Manage Google Colab drive
from google.colab import drive
drive.mount('/content/drive')


# General purpose libraries
import numpy as np
np.random.seed(42)
import pandas as pd
import re
import gensim
from collections import Counter
import os
os.environ['OMP_NUM_THREADS'] = '4' # Control the number of threads used by libraries utilizing OpenMP


# Data processing and machine learning functionalities
from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold


# Deep Learning libraries
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Conv1D, Embedding, SpatialDropout1D, concatenate
from keras.layers import GRU, LSTM,Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
# from keras.layers import CuDNNLSTM, CuDNNGRU
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
from keras import optimizers
from keras.layers import Lambda
from keras import backend as K


# Natural Language Processing libraries
from unidecode import unidecode
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
eng_stopwords = set(stopwords.words("english"))



# Handling warnings
import warnings
warnings.filterwarnings('ignore')

# Manual Garbage Collection
import gc


Mounted at /content/drive


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Dataset path configuration
input_dir = '/content/drive/MyDrive/Neural_networks/Dataset/original/'
output_dir = '/content/drive/MyDrive/Neural_networks/Dataset/Processed/'

In [ ]:
print(os.listdir(input_dir))
print(os.listdir(output_dir))

['train.csv', 'test.csv', 'sample_submission.csv', 'test_labels.csv']
['train_processed.csv', 'test_processed.csv']


this will turn all the bad words to standard form.for example terms like fuckedd will convert to fucked

In [ ]:
toxic_indicator_words = [
    'fuck', 'fucking', 'fucked', 'fuckin', 'fucka', 'fucker', 'fucks', 'fuckers',
    'fck', 'fcking', 'fcked', 'fckin', 'fcker', 'fcks',
    'fuk', 'fuking', 'fuked', 'fukin', 'fuker', 'fuks', 'fukers',
    'fk', 'fking', 'fked', 'fkin', 'fker', 'fks',
    'shit', 'shitty', 'shite',
    'stupid', 'stupids',
    'idiot', 'idiots',
    'suck', 'sucker', 'sucks', 'sucka', 'sucked', 'sucking',
    'ass', 'asses', 'asshole', 'assholes', 'ashole', 'asholes',
    'gay', 'gays',
    'niga', 'nigga', 'nigar', 'niggar', 'niger', 'nigger',
    'monster', 'monsters',
    'loser', 'losers',
    'nazi', 'nazis',
    'cock', 'cocks', 'cocker', 'cockers',
    'faggot', 'faggy',
]

toxic_indicator_words_sets = set(toxic_indicator_words)

def _get_toxicIndicator_transformers():
    toxicIndicator_transformers = dict()
    for word in toxic_indicator_words:
        print(f"\nProcessing word: {word}")  # Display the word being processed
        tmp_1 = []
        for c in word:
            print(f"Current character: {c}")
            if len(tmp_1) > 0:
                tmp_2 = []
                for pre in tmp_1:
                    new_comb1 = pre + c
                    new_comb2 = pre + c + c
                    tmp_2.append(new_comb1)  # Add new combination with single character
                    tmp_2.append(new_comb2)  # Add new combination with repeated character
                    print(f"New combinations: {new_comb1}, {new_comb2}")  # Show new combinations
                tmp_1 = tmp_2
            else:
                single_c = c
                double_c = c + c
                tmp_1.append(single_c)  # Add single character
                tmp_1.append(double_c)  # Add double characters
                print(f"Initial combinations: {single_c}, {double_c}")  # Show initial combinations

        # Map each variant to the base word
        for variant in tmp_1:
            toxicIndicator_transformers[variant] = word.lower() # Normalize the base word to lowercase
        print(f"Final mapped variants for '{word}': {set(tmp_1)}")  # Display all unique combinations for current word

    return toxicIndicator_transformers


toxicIndicator_transformers = _get_toxicIndicator_transformers()


def revise_toxic_variations(t, transformers):
    words = t.split()
    revised_words = [transformers.get(word, word) for word in words]
    return ' '.join(revised_words)


Processing word: fuck
Current character: f
Initial combinations: f, ff
Current character: u
New combinations: fu, fuu
New combinations: ffu, ffuu
Current character: c
New combinations: fuc, fucc
New combinations: fuuc, fuucc
New combinations: ffuc, ffucc
New combinations: ffuuc, ffuucc
Current character: k
New combinations: fuck, fuckk
New combinations: fucck, fucckk
New combinations: fuuck, fuuckk
New combinations: fuucck, fuucckk
New combinations: ffuck, ffuckk
New combinations: ffucck, ffucckk
New combinations: ffuuck, ffuuckk
New combinations: ffuucck, ffuucckk
Final mapped variants for 'fuck': {'ffuck', 'ffuckk', 'fuuck', 'ffuucck', 'ffuuck', 'ffuucckk', 'fuck', 'ffuuckk', 'ffucck', 'fuucckk', 'ffucckk', 'fuuckk', 'fucckk', 'fuucck', 'fuckk', 'fucck'}

Processing word: fucking
Current character: f
Initial combinations: f, ff
Current character: u
New combinations: fu, fuu
New combinations: ffu, ffuu
Current character: c
New combinations: fuc, fucc
New combinations: fuuc, fuucc
New

In [ ]:
def print_transformer_mappings(transformers):
    for variant, base_word in transformers.items():
        print(f"Variant: {variant} -> Base Word: {base_word}")

# Call the function to print the mappings
print_transformer_mappings(toxicIndicator_transformers)

Variant: fuck -> Base Word: fuck
Variant: fuckk -> Base Word: fuck
Variant: fucck -> Base Word: fuck
Variant: fucckk -> Base Word: fuck
Variant: fuuck -> Base Word: fuck
Variant: fuuckk -> Base Word: fuck
Variant: fuucck -> Base Word: fuck
Variant: fuucckk -> Base Word: fuck
Variant: ffuck -> Base Word: fuck
Variant: ffuckk -> Base Word: fuck
Variant: ffucck -> Base Word: fuck
Variant: ffucckk -> Base Word: fuck
Variant: ffuuck -> Base Word: fuck
Variant: ffuuckk -> Base Word: fuck
Variant: ffuucck -> Base Word: fuck
Variant: ffuucckk -> Base Word: fuck
Variant: fucking -> Base Word: fucking
Variant: fuckingg -> Base Word: fucking
Variant: fuckinng -> Base Word: fucking
Variant: fuckinngg -> Base Word: fucking
Variant: fuckiing -> Base Word: fucking
Variant: fuckiingg -> Base Word: fucking
Variant: fuckiinng -> Base Word: fucking
Variant: fuckiinngg -> Base Word: fucking
Variant: fuckking -> Base Word: fucking
Variant: fuckkingg -> Base Word: fucking
Variant: fuckkinng -> Base Word: fu

just to demonstrate above converter

In [ ]:
def normalize_text(text, transformers):
    words = text.split()
    # Ensure dictionary lookup keys are in the correct case
    normalized_words = [transformers.get(word.lower(), word) for word in words]
    return ' '.join(normalized_words)

# Lowercased to match the dictionary keys expected case
sample_text = "This is fuckingg  amazing with no fuckinngg mistakes fuckedd up!"
normalized_text = normalize_text(sample_text.lower(), toxicIndicator_transformers)
print(normalized_text)

this is fucking amazing with no fucking mistakes fucked up!


preprocessing section

In [ ]:
### Preprocessing section starts here ###

# Compile regex for number replacement and removal of special characters
replace_numbers = re.compile(r'\d+', re.IGNORECASE)
special_character_removal = re.compile(r'[^A-Za-z\.\-\?\!\,\#\@\% ]',re.IGNORECASE)


# Set key constants for data columns
COMMENT_COL = 'comment_text'
ID_COL = 'id'

# redundancy words and their right formats
redundancy_rightFormat = {
    'ckckck': 'cock',
    'fuckfuck': 'fuck',
    'lolol': 'lol',
    'lollol': 'lol',
    'pussyfuck':'fuck',
    'gaygay': 'gay',
    'haha': 'ha',
    'sucksuck': 'suck'}

redundancy = set(redundancy_rightFormat.keys())



deny_origin = {
    "you're": ['you', 'are'],
    "i'm": ['i', 'am'],
    "he's": ['he', 'is'],
    "she's": ['she', 'is'],
    "it's": ['it', 'is'],
    "they're": ['they', 'are'],
    "can't": ['can', 'not'],
    "couldn't": ['could', 'not'],
    "don't": ['do', 'not'],
    "don;t": ['do', 'not'],
    "didn't": ['did', 'not'],
    "doesn't": ['does', 'not'],
    "isn't": ['is', 'not'],
    "wasn't": ['was', 'not'],
    "aren't": ['are', 'not'],
    "weren't": ['were', 'not'],
    "won't": ['will', 'not'],
    "wouldn't": ['would', 'not'],
    "hasn't": ['has', 'not'],
    "haven't": ['have', 'not'],
    "what's": ['what', 'is'],
    "that's": ['that', 'is'],
}
denies = set(deny_origin.keys())


class CommProcess(object):
    @staticmethod
    def clean_text(t):
        t = re.sub(r"[^A-Za-z0-9,!?*.;’´'\/]", " ", t)
        t = replace_numbers.sub(" ", t)
        t = t.lower()
        t = re.sub(r",", " ", t)
        t = re.sub(r"’", "'", t)
        t = re.sub(r"´", "'", t)
        t = re.sub(r"\.", " ", t)
        t = re.sub(r"!", " ! ", t)
        t = re.sub(r"\?", " ? ", t)
        t = re.sub(r"\/", " ", t)
        return t

    @staticmethod
    def revise_deny(t):
        ret = []
        for word in t.split():
            if word in denies:
                ret.append(deny_origin[word][0])
                ret.append(deny_origin[word][1])
            else:
                ret.append(word)
        ret = ' '.join(ret)
        ret = re.sub("'", " ", ret)
        ret = re.sub(r";", " ", ret)
        return ret

    @staticmethod
    def revise_star(t):
        ret = []
        for word in t.split():
            if ('*' in word) and (re.sub('\*', '', word) in toxic_indicator_words_sets):
                word = re.sub('\*', '', word)
            ret.append(word)
        ret = re.sub('\*', ' ', ' '.join(ret))
        return ret

    @staticmethod
    def revise_triple_and_more_letters(t):
        for letter in 'abcdefghijklmnopqrstuvwxyz':
            reg = letter + "{2,}"
            t = re.sub(reg, letter + letter, t)
        return t

    @staticmethod
    def revise_toxic_variations(t):
        t = revise_toxic_variations(t, toxicIndicator_transformers)
        # Additional processing methods as defined
        return t

    @staticmethod
    def revise_redundancy_words(t):
        ret = []
        for word in t.split(' '):
            for redu in redundancy:
                if redu in word:
                    word = redundancy_rightFormat[redu]
                    break
            ret.append(word)
        return ' '.join(ret)

    @staticmethod
    def fill_na(t):
        if t.strip() == '':
            return 'NA'
        return t

def clean_text(x):
    x_ascii = unidecode(x)
    x_clean = special_character_removal.sub('',x_ascii)
    return x_clean

def execute_comm_process(df):
    comm_process_pipeline = [
        CommProcess.clean_text,
        CommProcess.revise_deny,
        CommProcess.revise_star,
        CommProcess.revise_triple_and_more_letters,
        CommProcess.revise_toxic_variations,
        CommProcess.revise_redundancy_words,
        CommProcess.fill_na,
    ]
    for cp in comm_process_pipeline:
        df[COMMENT_COL] = df[COMMENT_COL].apply(cp)
    return df



# Process whole train data
print('Comm processing whole train data')
df_train = pd.read_csv(input_dir + 'train.csv')
df_train = execute_comm_process(df_train)
df_train['clean_text'] = df_train['comment_text'].apply(lambda x: clean_text(str(x)))
df_train.to_csv(output_dir + 'train_processed.csv', index=False)

# Process test data
print('Comm processing test data')
df_test = pd.read_csv(input_dir + 'test.csv')
df_test = execute_comm_process(df_test)
df_test['clean_text'] = df_test['comment_text'].apply(lambda x: clean_text(str(x)))
df_test.to_csv(output_dir + 'test_processed.csv', index=False)



train = pd.read_csv(output_dir + 'train_processed.csv')
test = pd.read_csv(output_dir + 'test_processed.csv')

Comm processing whole train data
Comm processing test data


In [ ]:
X_train = train['clean_text'].fillna("something").values
y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
X_test = test['clean_text'].fillna("something").values

below to view dataset to compare

In [ ]:
train_original = pd.read_csv(input_dir + 'train.csv')

# Compare original and processed text
comparison_df = pd.DataFrame({
    'Original Text': train_original['comment_text'].head(5),
    'Processed Text': train['clean_text'].head(5)
})

# Use the custom display function to present the comparison DataFrame
display_dataframe(comparison_df, num_rows=5)

DataFrame Overview:


,Original Text,Processed Text
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",explanation why the edits made under my username hardcore metallica fan were reverted ? they were not vandalisms just closure on some gas after i voted at new york dolls fac and please do not remove the template from the talk page since i am retired now
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",d aww ! he matches this background colour i am seemingly stuck with thanks talk january utc
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",hey man i am really not trying to edit war it is just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,more i can not make any real suggestions on improvement i wondered if the section statistics should be later on or a subsection of types of accidents i think the references may need tidying so that they are all in the exact same format ie date format etc i can do that later on if no one else does first if you have any preferences for formatting style on references or want to do it yourself please let me know there appears to be a backlog on articles for review so i guess there may be a delay until a reviewer turns up it is listed in the relevant form eg wikipedia good article nominations transport
4,"You, sir, are my hero. Any chance you remember what page that's on?",you sir are my hero any chance you remember what page that is on ?


In [ ]:
# Convert arrays to pandas DataFrame for display
df_X_train = pd.DataFrame(X_train, columns=['X_train'])
df_y_train = pd.DataFrame(y_train, columns=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'])
df_X_test = pd.DataFrame(X_test, columns=['X_test'])

# Display the DataFrames using the custom function
display_dataframe(df_X_train, num_rows=5)
display_dataframe(df_y_train, num_rows=5)
display_dataframe(df_X_test, num_rows=5)

DataFrame Overview:


,X_train
0,explanation why the edits made under my username hardcore metallica fan were reverted ? they were not vandalisms just closure on some gas after i voted at new york dolls fac and please do not remove the template from the talk page since i am retired now
1,d aww ! he matches this background colour i am seemingly stuck with thanks talk january utc
2,hey man i am really not trying to edit war it is just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info
3,more i can not make any real suggestions on improvement i wondered if the section statistics should be later on or a subsection of types of accidents i think the references may need tidying so that they are all in the exact same format ie date format etc i can do that later on if no one else does first if you have any preferences for formatting style on references or want to do it yourself please let me know there appears to be a backlog on articles for review so i guess there may be a delay until a reviewer turns up it is listed in the relevant form eg wikipedia good article nominations transport
4,you sir are my hero any chance you remember what page that is on ?


DataFrame Overview:


,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0


DataFrame Overview:


,X_test
0,yo bitch ja rule is more succesful then you ll ever be whats up with you and hating you sad mofuckas i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me ja rule is about pride in da music man dont diss that shit on him and nothin is wrong bein like tupac he was a brother too fuckin white boys get things right next time
1,from rfc the title is fine as it is imo
2,sources zawe ashton on lapland
3,if you have a look back at the source the information i updated was the correct form i can only guess the source hadn t updated i shall update the information once again but thank you for your message
4,i do not anonymously edit articles at all


In [ ]:
# Feature Engineering
def add_features(df):

    df['comment_text'] = df['comment_text'].apply(lambda x:str(x))
    df['total_length'] = df['comment_text'].apply(len)
    df['capitals'] = df['comment_text'].apply(lambda comment: sum(1 for c in comment if c.isupper()))
    df['caps_vs_length'] = df.apply(lambda row: float(row['capitals'])/float(row['total_length']),
                                axis=1)
    df['num_words'] = df.comment_text.str.count('\S+')
    df['num_unique_words'] = df['comment_text'].apply(lambda comment: len(set(w for w in comment.split())))
    df['words_vs_unique'] = df['num_unique_words'] / df['num_words']

    return df

train = add_features(train)
test = add_features(test)

#just to see the dataset
print("Train Dataset Feature Overview:")
display_dataframe(train)
print("\nTest Dataset Feature Overview:")
display_dataframe(test, num_rows=3)


features = train[['caps_vs_length', 'words_vs_unique']].fillna(0)
test_features = test[['caps_vs_length', 'words_vs_unique']].fillna(0)

# Normalize Features
ss = StandardScaler()
ss.fit(np.vstack((features, test_features)))
features = ss.transform(features)
test_features = ss.transform(test_features)

Train Dataset Feature Overview:
DataFrame Overview:


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean_text,total_length,capitals,caps_vs_length,num_words,num_unique_words,words_vs_unique
0,0000997932d777bf,explanation why the edits made under my username hardcore metallica fan were reverted ? they were not vandalisms just closure on some gas after i voted at new york dolls fac and please do not remove the template from the talk page since i am retired now,0,0,0,0,0,0,explanation why the edits made under my username hardcore metallica fan were reverted ? they were not vandalisms just closure on some gas after i voted at new york dolls fac and please do not remove the template from the talk page since i am retired now,253,0,0.0,47,42,0.893617
1,000103f0d9cfb60f,d aww ! he matches this background colour i am seemingly stuck with thanks talk january utc,0,0,0,0,0,0,d aww ! he matches this background colour i am seemingly stuck with thanks talk january utc,91,0,0.0,17,17,1.000000
2,000113f07ec002fd,hey man i am really not trying to edit war it is just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info,0,0,0,0,0,0,hey man i am really not trying to edit war it is just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info,231,0,0.0,44,40,0.909091
3,0001b41b1c6bb37e,more i can not make any real suggestions on improvement i wondered if the section statistics should be later on or a subsection of types of accidents i think the references may need tidying so that they are all in the exact same format ie date format etc i can do that later on if no one else does first if you have any preferences for formatting style on references or want to do it yourself please let me know there appears to be a backlog on articles for review so i guess there may be a delay until a reviewer turns up it is listed in the relevant form eg wikipedia good article nominations transport,0,0,0,0,0,0,more i can not make any real suggestions on improvement i wondered if the section statistics should be later on or a subsection of types of accidents i think the references may need tidying so that they are all in the exact same format ie date format etc i can do that later on if no one else does first if you have any preferences for formatting style on references or want to do it yourself please let me know there appears to be a backlog on articles for review so i guess there may be a delay until a reviewer turns up it is listed in the relevant form eg wikipedia good article nominations transport,604,0,0.0,116,82,0.706897
4,0001d958c54c6e35,you sir are my hero any chance you remember what page that is on ?,0,0,0,0,0,0,you sir are my hero any chance you remember what page that is on ?,66,0,0.0,15,14,0.933333



Test Dataset Feature Overview:
DataFrame Overview:


,id,comment_text,clean_text,total_length,capitals,caps_vs_length,num_words,num_unique_words,words_vs_unique
0,00001cee341fdb12,yo bitch ja rule is more succesful then you ll ever be whats up with you and hating you sad mofuckas i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me ja rule is about pride in da music man dont diss that shit on him and nothin is wrong bein like tupac he was a brother too fuckin white boys get things right next time,yo bitch ja rule is more succesful then you ll ever be whats up with you and hating you sad mofuckas i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me ja rule is about pride in da music man dont diss that shit on him and nothin is wrong bein like tupac he was a brother too fuckin white boys get things right next time,358,0,0.0,75,62,0.826667
1,0000247867823ef7,from rfc the title is fine as it is imo,from rfc the title is fine as it is imo,39,0,0.0,10,9,0.900000
2,00013b17ad220c46,sources zawe ashton on lapland,sources zawe ashton on lapland,30,0,0.0,5,5,1.000000


In [ ]:
# Display the first few rows of the features and test_features arrays
print("Training Features (first 5 rows):")
print(features[:5])  # Display the first five rows of training features

print("\nTest Features (first 5 rows):")
print(test_features[:5])  # Display the first five rows of test features

Training Features (first 5 rows):
[[ 0.          0.4950471 ]
 [ 0.          1.17117593]
 [ 0.          0.59339311]
 [ 0.         -0.69167557]
 [ 0.          0.74746853]]

Test Features (first 5 rows):
[[ 0.          0.06953669]
 [ 0.          0.53561483]
 [ 0.          1.17117593]
 [ 0.         -0.45846792]
 [ 0.          1.17117593]]


In [ ]:


# For best score (Public: 9869, Private: 9865), change to max_features = 283759, maxlen = 900
max_features = 10000
maxlen = 50

# Prepare Sequences for Model
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train) + list(X_test))
X_train_sequence = tokenizer.texts_to_sequences(X_train)
X_test_sequence = tokenizer.texts_to_sequences(X_test)

x_train = sequence.pad_sequences(X_train_sequence, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test_sequence, maxlen=maxlen)
print(len(tokenizer.word_index))

# Load the FastText Web Crawl vectors
EMBEDDING_FILE_FASTTEXT="/content/drive/MyDrive/Neural_networks/EmbeddingModels/FastText/crawl-300d-2M.vec"
EMBEDDING_FILE_TWITTER="/content/drive/MyDrive/Neural_networks/EmbeddingModels/GloVe/glove.twitter.27B.200d.txt"
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index_ft = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE_FASTTEXT,encoding='utf-8'))
embeddings_index_tw = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE_TWITTER,encoding='utf-8'))

spell_model = gensim.models.KeyedVectors.load_word2vec_format(EMBEDDING_FILE_FASTTEXT)

# This code is  based on: Spellchecker using Word2vec by CPMP
# https://www.kaggle.com/cpmpml/spell-checker-using-word2vec

words = spell_model.index_to_key

w_rank = {}
for i,word in enumerate(words):
    w_rank[word] = i

WORDS = w_rank

# Use fast text as vocabulary
def words(text): return re.findall(r'\w+', text.lower())

def P(word):
    "Probability of `word`."
    # use inverse of rank as proxy
    # returns 0 if the word isn't in the dictionary
    return - WORDS.get(word, 0)

def correction(word):
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def singlify(word):
    return "".join([letter for i,letter in enumerate(word) if i == 0 or letter != word[i-1]])

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words,501))

something_tw = embeddings_index_tw.get("something")
something_ft = embeddings_index_ft.get("something")

something = np.zeros((501,))
something[:300,] = something_ft
something[300:500,] = something_tw
something[500,] = 0

def all_caps(word):
    return len(word) > 1 and word.isupper()

def embed_word(embedding_matrix,i,word):
    embedding_vector_ft = embeddings_index_ft.get(word)
    if embedding_vector_ft is not None:
        if all_caps(word):
            last_value = np.array([1])
        else:
            last_value = np.array([0])
        embedding_matrix[i,:300] = embedding_vector_ft
        embedding_matrix[i,500] = last_value
        embedding_vector_tw = embeddings_index_tw.get(word)
        if embedding_vector_tw is not None:
            embedding_matrix[i,300:500] = embedding_vector_tw


# Fasttext vector is used by itself if there is no glove vector but not the other way around.
for word, i in word_index.items():

    if i >= max_features: continue

    if embeddings_index_ft.get(word) is not None:
        embed_word(embedding_matrix,i,word)
    else:
        # change to > 20 for better score.
        if len(word) > 0:
            embedding_matrix[i] = something
        else:
            word2 = correction(word)
            if embeddings_index_ft.get(word2) is not None:
                embed_word(embedding_matrix,i,word2)
            else:
                word2 = correction(singlify(word))
                if embeddings_index_ft.get(word2) is not None:
                    embed_word(embedding_matrix,i,word2)
                else:
                    embedding_matrix[i] = something

class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data
        self.max_score = 0
        self.not_better_count = 0

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=1)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))
            if (score > self.max_score):
                print("*** New High Score (previous: %.6f) \n" % self.max_score)
                model.save_weights("/content/drive/MyDrive/Neural_networks/Weights/best_weights.h5")
                self.max_score=score
                self.not_better_count = 0
            else:
                self.not_better_count += 1
                if self.not_better_count > 3:
                    print("Epoch %05d: early stopping, high score = %.6f" % (epoch,self.max_score))
                    self.model.stop_training = True
def get_model(features,clipvalue=1.,num_filters=40,dropout=0.5,embed_size=501):
    features_input = Input(shape=(features.shape[1],))
    inp = Input(shape=(maxlen, ))

    # Layer 1: concatenated fasttext and glove twitter embeddings.
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)

    # Uncomment for best result
    # Layer 2: SpatialDropout1D(0.5)
    #x = SpatialDropout1D(dropout)(x)

    # Uncomment for best result
    # Layer 3: Bidirectional CuDNNLSTM
    #x = Bidirectional(LSTM(num_filters, return_sequences=True))(x)


    # Layer 4: Bidirectional CuDNNGRU
    x, x_h, x_c = Bidirectional(GRU(num_filters, return_sequences=True, return_state = True))(x)

    # Layer 5: A concatenation of the last state, maximum pool, average pool and
    # two features: "Unique words rate" and "Rate of all-caps words"
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)

    x = concatenate([avg_pool, x_h, max_pool,features_input])

    # Layer 6: output dense layer.
    outp = Dense(6, activation="sigmoid")(x)

    model = Model(inputs=[inp,features_input], outputs=outp)
    adam = optimizers.adam(clipvalue=clipvalue)
    model.compile(loss='binary_crossentropy',
                  optimizer=adam,
                  metrics=['accuracy'])
    return model

def get_model(features,clipvalue=1.,num_filters=40,dropout=0.5,embed_size=501):
    features_input = Input(shape=(features.shape[1],))
    inp = Input(shape=(maxlen, ))

    # Layer 1: concatenated fasttext and glove twitter embeddings.
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)

    # Uncomment for best result
    # Layer 2: SpatialDropout1D(0.5)
    #x = SpatialDropout1D(dropout)(x)

    # Uncomment for best result
    # Layer 3: Bidirectional CuDNNLSTM
    #x = Bidirectional(LSTM(num_filters, return_sequences=True))(x)


    # Layer 4: Bidirectional CuDNNGRU
    x, x_h, x_c = Bidirectional(GRU(num_filters, return_sequences=True, return_state = True))(x)

    # Layer 5: A concatenation of the last state, maximum pool, average pool and
    # two features: "Unique words rate" and "Rate of all-caps words"
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)

    x = concatenate([avg_pool, x_h, max_pool,features_input])

    # Layer 6: output dense layer.
    outp = Dense(6, activation="sigmoid")(x)

    model = Model(inputs=[inp,features_input], outputs=outp)

    from keras.optimizers import Adam
    adam_optimizer = Adam(clipvalue=clipvalue)

    model.compile(loss='binary_crossentropy', optimizer=adam_optimizer, metrics=['accuracy'])
    return model

model = get_model(features)

batch_size = 32

# Used epochs=100 with early exiting for best score.
epochs = 1
gc.collect()
K.clear_session()

# Change to 10
num_folds = 2 #number of folds

predict = np.zeros((test.shape[0],6))

# Uncomment for out-of-fold predictions
#scores = []
#oof_predict = np.zeros((train.shape[0],6))

kf = KFold(n_splits=num_folds, shuffle=True, random_state=239)

for train_index, test_index in kf.split(x_train):

    kfold_y_train,kfold_y_test = y_train[train_index], y_train[test_index]
    kfold_X_train = x_train[train_index]
    kfold_X_features = features[train_index]
    kfold_X_valid = x_train[test_index]
    kfold_X_valid_features = features[test_index]

    gc.collect()
    K.clear_session()

    model = get_model(features)

    ra_val = RocAucEvaluation(validation_data=([kfold_X_valid,kfold_X_valid_features], kfold_y_test), interval = 1)

    model.fit([kfold_X_train,kfold_X_features], kfold_y_train, batch_size=batch_size, epochs=epochs, verbose=1,
             callbacks = [ra_val])
    gc.collect()

    #model.load_weights(bst_model_path)
    model.load_weights("/content/drive/MyDrive/Neural_networks/Weights/best_weights.h5")

    predict += model.predict([x_test,test_features], batch_size=batch_size,verbose=1) / num_folds

    #gc.collect()
    # uncomment for out of fold predictions
    #oof_predict[test_index] = model.predict([kfold_X_valid, kfold_X_valid_features],batch_size=batch_size, verbose=1)
    #cv_score = roc_auc_score(kfold_y_test, oof_predict[test_index])

    #scores.append(cv_score)
    #print('score: ',cv_score)

print("Done")
#print('Total CV score is {}'.format(np.mean(scores)))


sample_submission = pd.read_csv("/content/drive/MyDrive/Neural_networks/Dataset/original/sample_submission.csv")
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
sample_submission[class_names] = predict
sample_submission.to_csv('/content/drive/MyDrive/Neural_networks/model_9872_baseline_submission.csv',index=False)

# uncomment for out of fold predictions
#oof = pd.DataFrame.from_dict({'id': train['id']})
#for c in class_names:
#    oof[c] = np.zeros(len(train))
#
#oof[class_names] = oof_predict
#for c in class_names:
#    oof['prediction_' +c] = oof[c]
#oof.to_csv('oof-model_9872_baseline_submission.csv', index=False)







283666
2494/2494 [==============================] - 11s 4ms/step

 ROC-AUC - epoch: 1 - score: 0.979240 

*** New High Score (previous: 0.000000) 

2494/2494 [==============================] - 10s 4ms/step

 ROC-AUC - epoch: 1 - score: 0.979351 

*** New High Score (previous: 0.000000) 

4787/4787 [==============================] - 18s 4ms/step
Done
